In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable 
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [113]:
def batch_data(size, dataset):
    in_data, out_data = zip(*dataset)
    in_data_vec = []
    for i in list(in_data):
        vec = []
        for j in range(10):
            vec.append(int(i[j]))
        in_data_vec.append(vec)
    out_data = list(out_data)
    in_arr = np.array(in_data_vec)
    out_arr = np.array(out_data)
    
    in_arr = in_arr.reshape(-1, size, 10)
    in_arr = in_arr.astype(np.float32)

    out_arr = out_arr.reshape(-1, size, 1)
    out_arr = out_arr.astype(np.float32)
    print(in_arr.shape, out_arr.shape)
    batched_data = []
    for i in range(in_arr.shape[0]):
        batched_data.append((torch.from_numpy(in_arr[i]), torch.from_numpy(out_arr[i])))
    return batched_data

In [114]:
LOG_INTERVAL = 10 
dataset = [("{:010b}".format(i), i) for i in range(1024)]
random.shuffle(dataset)
train_set = dataset[:1000]
test_set = dataset[1000:]

train_loader = batch_data(20, train_set)
test_loader = batch_data(24, test_set)

def train(model, optimizer, loss_func, epoch, training_history):
    model.train()                                            #Set training mode 
    for batch, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)      #Make data and correct answers variables in the Network
        optimizer.zero_grad()                                #zero the gradients
        output = model(data)                                 #classify the data
        loss = loss_func(output, target)                     #calculate the loss
        loss.backward()                                      #propogate the weight updates through the network
        optimizer.step()
        if batch % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch * len(data), len(train_loader), 100. * batch / len(train_loader), loss.data[0]))
            training_history.append(((len(train_loader) * epoch) + batch * len(data), loss.data[0]))


def test(model, loss_func, epoch, test_loss_history, test_accuracy_history):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target) #Make data and correct answers variables in the Network
        output = model(data)                                           #classify the data
        test_loss += loss_func(output, target).data[0]                 #calculate loss
        pred = output.data.max(1)[1]                                   #get predictions for the batch using argmax
        #correct += pred.eq(target.data).cpu().sum()                    #total correct anwsers 
    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))
    test_loss_history.append((epoch, test_loss))
    test_accuracy_history.append((epoch, 100. * correct / len(test_loader)))
    
def learn(model, optimizer, loss_func):
    training_loss = []
    test_loss = []
    test_accuracy = []
    for e in range(EPOCHS):
        train(model, optimizer, loss_func, e + 1, training_loss)
        test(model, loss_func, e + 1, test_loss, test_accuracy)
    return (training_loss, test_loss, test_accuracy)



def visualize_learning(training_loss, test_loss, test_accuracy):
    f1 = plt.figure()
    f2 = plt.figure()
    f3 = plt.figure()
    ax1 = f1.add_subplot(111)
    ax2 = f2.add_subplot(111)
    ax3 = f3.add_subplot(111)

    training_loss_batch, training_loss_values = zip(*training_loss)
    ax1.plot(training_loss_batch, training_loss_values)
    ax1.set_title('Training Loss')
    ax1.set_xlabel("Batch")
    ax1.set_ylabel("Loss")


    test_loss_epoch, test_loss_values = zip(*test_loss)
    ax2.plot(test_loss_epoch, test_loss_values)
    ax2.set_title("Testing Loss")
    ax2.set_xlabel("Batch")
    ax2.set_ylabel("Loss")

    test_accuracy_epoch, test_accuracy_values = zip(*test_accuracy)
    ax3.plot(test_accuracy_epoch, test_accuracy_values)
    ax3.set_title("Testing Accuracy")
    ax3.set_xlabel("Batch")
    ax3.set_ylabel("Accuracy (%)")

    plt.show()

def classify(model, img):
    img = Variable(img, volatile=True)
    output = model(img)
    return output.data.max(1)[1]

def classify_an_example(model):
    img = next(iter(test_loader))[0]
    img_np = img.numpy()[0]
    plt.imshow(img_np.reshape(28,28))
    print()
    print("The image is probably: {}".format(classify(model, img)[0]))
    print()

(50, 20, 10) (50, 20, 1)
(1, 24, 10) (1, 24, 1)


In [115]:
### train MLP
class NMLP(nn.Module):
    '''
    3 Hidden Layers, Narrow Model Width for the Hidden Layers (total hidden neurons 2000)
    '''
    def __init__(self):
        super(NMLP, self).__init__()
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 25)
        self.fc4 = nn.Linear(25, 10)
        self.fc5 = nn.Linear(10,1)
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return F.log_softmax(x, dim=1)

In [116]:
EPOCHS = 1500                #@param {type:"integer"}           #Number of times to go through the data set
SGD_MOMENTUM = 0.5        #@param {type:"number"}            #How much it takes to change the direction of the gradient
LEARNING_RATE = 0.001     #@param {type:"number"}            #How far each update pushes the weights
narrow_mlp_model = NMLP()
mlp_optimizer = optim.SGD(narrow_mlp_model.parameters(), lr=LEARNING_RATE, momentum=SGD_MOMENTUM)
print(narrow_mlp_model)
NMLP_TRAINING_LOSS, NMLP_TEST_LOSS, NMLP_TEST_ACCURACY = learn(narrow_mlp_model, mlp_optimizer, nn.MSELoss())

NMLP(
  (fc1): Linear(in_features=10, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=25, bias=True)
  (fc4): Linear(in_features=25, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=1, bias=True)
)
Train Epoch: 1 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 2 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 2 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 2 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 2 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 2 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 3 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 3 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 3 [400/50 (

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 7 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 7 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 7 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 7 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 8 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 8 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 8 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 8 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 8 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 9 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 9 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 9 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 9 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 9 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 10 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 10 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 10 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 10 [600/50 (60%)]	Loss: 40136


Test set: Average loss: 332177.1562

Train Epoch: 36 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 36 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 36 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 36 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 36 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 37 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 37 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 37 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 37 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 37 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 38 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 38 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 38 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 38 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 38 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 39 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 39 [200/50 (20%)]	Loss: 324084.

Train Epoch: 65 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 66 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 66 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 66 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 66 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 66 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 67 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 67 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 67 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 67 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 67 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 68 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 68 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 68 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 68 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 68 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 69 [0/50 (0%)]	Loss: 398070.

...

Train Epoch: 1371 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1371 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1371 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1371 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1372 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1372 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1372 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1372 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1372 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1373 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1373 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1373 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1373 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1373 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1374 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1374 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1374 [400/50 (40%)]	Loss: 400089.9

Train Epoch: 1399 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1399 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1399 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1399 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1400 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1400 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1400 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1400 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1400 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1401 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1401 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1401 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1401 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1401 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1402 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1402 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1402 [400/50 (40%)]	Loss: 400089.9

Train Epoch: 1426 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1426 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1427 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1427 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1427 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1427 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1427 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1428 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1428 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1428 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1428 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1428 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1429 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1429 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1429 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1429 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1429 [800/50 (80%)]	Loss: 550711.1

Train Epoch: 1458 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1458 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1459 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1459 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1459 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1459 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1459 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1460 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1460 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1460 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1460 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1460 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1461 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1461 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1461 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1461 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1461 [800/50 (80%)]	Loss: 550711.1

Train Epoch: 1489 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1490 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1490 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1490 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1490 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1490 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1491 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1491 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1491 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1491 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1491 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 1492 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1492 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1492 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1492 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1492 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epo

In [118]:
import convert_pyt_to_adj_list as converter

In [119]:
converter.convert_sd_to_adj("narrow_binary_converter", narrow_mlp_model)

AttributeError: 'NMLP' object has no attribute 'keys'